## Dog Cat Classification

### 1 - Package import 

In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import os
import random

import tensorflow as tf
import tensorflow.keras.layers as tfl

from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

### 2 - Data Import

In [24]:
train_path = '../input/dogs-vs-cats/train.zip'
test_path = '../input/dogs-vs-cats/test1.zip'

destination = '/kaggle/files/images'

from zipfile import ZipFile as zipper
with zipper(train_path, 'r') as zipp:
    zipp.extractall(destination)
    
with zipper(test_path, 'r') as zipp:
    zipp.extractall(destination)

In [25]:
train = pd.DataFrame({'file': os.listdir('/kaggle/files/images/train')})

categories = []
for i in os.listdir('/kaggle/files/images/train'):
    if 'dog' in i:
        categories.append(1)
    else:
        categories.append(0)
        
train['categories'] = categories

test = pd.DataFrame({'file': os.listdir('/kaggle/files/images/test1')})

### 3 - See an example 

In [26]:
def example_im(index):
    '''
    Function to display an example image.
    
    index -- which image in the training set. 
    '''
    im = plt.imread('/kaggle/files/images/train/'+str(train['file'][index]))

    print(type(im))
    print(im.shape)
    print(type(im.shape))
    
    plt.imshow(im)
    plt.axis('off')
example_im(1)

### 4 - Data Preparation 

In [27]:
train['categories'].value_counts()

For our data, we have same number of cats and dogs and training data.

In [28]:
train.shape

The numeric varaible in `categories` columns is mapped to string for later image generation

The training data is not a particularly small dataset so we can use `10%` of its data as a `cross validation set`, and `random_state` is set to be 0 for reproductivity.

In [29]:
train['categories'] = train['categories'].replace({0: 'cat', 1: 'dog'})
train_set, val_set = train_test_split(train, test_size=0.1, random_state = 0)

In [30]:
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

batch_size = 64

train_generator = train_gen.flow_from_dataframe(
    dataframe = train_set,
    directory = destination + '/train/', # file path format
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size
)


validation_generator = val_gen.flow_from_dataframe(
    dataframe = val_set,
    directory = destination + '/train/',
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size
)

In [31]:
def mini_batch_example_plot(df):

    example_generator = train_gen.flow_from_dataframe(
        dataframe = df,
        directory = destination + '/train/',
        x_col = 'file',
        y_col = 'categories',
        class_mode = 'categorical',
        target_size = (224,224)
    )
    
    fig, ax  = plt.subplots(2,4,figsize=(12, 12))
    ax = ax.flatten()
    
    for i in range(8):
        X, Y = next(example_generator)
        image = X[0]
        ax[i].imshow(image)
        ax[i].axis('off')
    
    
mini_batch_example_plot(train_set)

### 5 - CNN model construction

In [65]:
from numpy.random import seed
seed(1)

tf.random.set_seed(2)

In [32]:
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation


In [33]:
# set constants
(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS) = (224,224,3)
IMAGE_SIZE  = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_SHAPE = (224,224,3)

In [34]:
def shallow_CNN_Model(image_shape, augmentation = False):
    '''
    This creates a shallow CNN model, the structure uses the idea from VGG-16
    '''
    if (augmentation == True):
        model = tf.keras.Sequential([RandomFlip("horizontal",input_shape = image_shape),
                   RandomRotation(0.1)])
    else: 
        model = tf.keras.Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape = image_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    return model

In [35]:
model = shallow_CNN_Model(IMAGE_SHAPE)
model_augmented = shallow_CNN_Model(IMAGE_SHAPE,True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_augmented.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

### 6 - Early stopping after seeing no decrease in loss over 5 epoches

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(patience=5)

In [37]:
def model_fitting(model, callbacks, epochs):
    '''
    This is a function used to fit generator, with customized input 
    model, callbacks, and epochs. 
    '''
    return model.fit_generator(
        train_generator, 
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = val_set.shape[0]//64,
        steps_per_epoch = train_set.shape[0]//64,
        callbacks = callbacks 
    )
    

### 7 - Fitting

In previous test, the dataset not being augmented performs worse than augmented dataset on validation set, showing a sign of overfitting in training set.  

In [38]:
# history_epoch50 = model_fitting(model, [earlystop], epochs = 50)

In [39]:
history_epoch50_augmented = model_fitting(model_augmented, [earlystop], epochs = 50)

In [45]:
model_augmented.save_weights('model_augmented_weights.h5')

In [46]:

def plotting_loss(history,upper_bound):

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 4))
    ax1.plot(history.history['loss'], color='b', label="Training loss")
    ax1.plot(history.history['val_loss'], color='r', label="validation loss")
    ax1.set_xticks(np.arange(1, upper_bound, 1))
    ax1.set_yticks(np.arange(0, 1, 0.1))

    ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    ax2.set_xticks(np.arange(1, upper_bound, 1))

    legend = plt.legend(loc='best', shadow=True)
    plt.tight_layout()
    plt.show()


In [47]:
# plotting_loss(history_epoch50, 50)

In [49]:
plotting_loss(history_epoch50_augmented, 40)